In [ ]:
%matplotlib inline
from pprint import pprint
from decimal import Decimal
import itertools
import metrics
import pandas

import simulate
import harvesting
import market
import withdrawal
import numpy

In [ ]:
import seaborn
from matplotlib import pyplot as plt
import matplotlib
seaborn.set(style="whitegrid")
seaborn.set_context('poster')

In [ ]:
series = market.Returns_US_1871()

In [ ]:
TYPE = 'Withdrawal (Real)'
m = {'Withdrawal (Nominal)': lambda x: float(x.withdraw_n),
     'Withdrawal (Real)' : lambda x: float(x.withdraw_r),
     'Portfolio (Nominal)': lambda x: float(x.portfolio_n),
     'Portfolio (Real)': lambda x: float(x.portfolio_r),
     'Bond %': lambda x: float(x.portfolio_post.bonds / (x.portfolio_post.bonds + x.portfolio_post.stocks))
}
lens = m[TYPE]
lens.__label__ = TYPE

def fn(x):
    return [lens(_) for _ in x]

In [ ]:
VPW_YEARS=40

def run_sim(year, key, length=30):
    h = {
        'tent': lambda p: harvesting.ParameterGlidepath(p, 0.75, 1, 30),
        'static-87': harvesting.make_rebalancer(0.87),
    }

    portfolio = (500000,500000)
    
    return simulate.withdrawals(series.iter_from(year),
                                 withdraw=lambda p, s: withdrawal.VPW(p, s, years_left=VPW_YEARS),
                                 years=length,
                                 portfolio=portfolio,
                                 harvesting=h[key])

In [ ]:
def cew_one(year):
    tent = run_sim(year, 'tent')
    static = run_sim(year, 'static-87')
    
    df = pandas.DataFrame({
        'tent' : [x.withdraw_r for x in tent],
        'static-87' : [x.withdraw_r for x in static],
    })
    return df.apply(metrics.cew)

def cew_all():
    df = pandas.DataFrame(index=range(1887, 2019 - 30 + 1), columns=['tent', 'static-87'])
    for i in range(1887, 2019 - 30 + 1):
        r = cew_one(i)
        df.loc[i] = r
    return df

In [ ]:
df = cew_all()

In [ ]:
df.head()

In [ ]:
(df[df['tent'] > df['static-87']]).head()

In [ ]:
a = df[df['static-87'] > df['tent']]
s = (a['static-87'] - a['tent']) / a['tent']
print(s.mean())
s.sort_values(ascending=False).head()
#len(s[s > 0.05])

In [ ]:
s = a['static-87'] - a['tent']
print(s.sort_values(ascending=False).head())
print(s.mean())
len(s[s>5000]) / len(s)

In [ ]:
a = df[df['tent'] > df['static-87']]
s = (a['tent'] - a['static-87'])
s.sort_values(ascending=False).mean()

In [ ]:
def chart_one(year):
    tent = run_sim(year, 'tent')
    static = run_sim(year, 'static-87')
    
    df = pandas.DataFrame({
        'tent' : [lens(x) for x in tent],
        'static-87' : [lens(x) for x in static],
    })
    g = seaborn.relplot(
            data=df,
            kind='line',
            aspect=2,
        )
    g.fig.autofmt_xdate()
    g.despine(left=True, bottom=True, offset=20)
    g.fig.suptitle(f'Retirement in {year}')
    return g

In [ ]:
chart_one(1929)
df.loc[1929]

In [ ]:
def one(year):
    tent = run_sim(year, 'tent')
    static = run_sim(year, 'static-87')
    
    df = pandas.DataFrame({
        'tent' : [lens(x) for x in tent],
        'static-87' : [lens(x) for x in static],
    })
    return df

In [ ]:
one(1929)